In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/commit/Dacon/nlp_review/train.csv')
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/commit/Dacon/nlp_review/test.csv')

In [ ]:
train.head()

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       25000 non-null  int64 
 1   reviews  25000 non-null  object
 2   target   25000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 586.1+ KB


#데이터 전처리

In [ ]:
X = train.reviews #training 데이터에서 피쳐 추출
y = train.target #training 데이터에서 타겟 추출

In [ ]:
!pip install konlpy
from konlpy.tag import Okt
okt=Okt()


tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
def token(dataframe):
    for sentence in dataframe['reviews']: # 전처리된 리뷰들을 하나씩 꺼내옵니다
        tokens = okt.morphs(sentence)
        tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
        tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
        dataframe["tokenized_stem"] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.

token(train)
token(test)

train.head() # 데이터 확인


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 41.1 MB/s eta 0:00:00


,id,reviews,target,tokenized_stem
0,0,조아요 처음구입 싸게햇어요,2,조아요 처음 구입 싸게 햇 어 요
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1,생각 보다 잘 안 돼요 매지 바른지 하루 밖에 안 됐는데 ㅠㅠ 25천원 가량 주고 ...
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2,디자인 은 괜찮은데 상품 이 금 이 가서 교환 했는데 두번째 받은 상품 도 까져있고...
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2,기전 에 이 제품 말고 이마트 트 레이더스 에서만 팔던 프리미엄 제품 을 사용 했었...
4,4,튼튼하고 손목을 잘 받쳐주네요~,5,튼튼하고 손목 을 잘 받쳐주네요 ~


##품사 태깅

In [ ]:
def postagging(dataframe):
    main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
    for sentence in dataframe['tokenized_stem']: # 리뷰들을 하나씩 가져옵니다
        pos = okt.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
        main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ('Adjective', #형용사
                                                                         'Noun', #명사
                                                                         'Adjective', #부사
                                                                         'Verb', #동사
                                                                         'Number',
                                                                         'KoreanParticle'
                                                                         )] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
        main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
        main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다
        dataframe["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.

postagging(train)
#postagging(test) ## test셋도 똑같이 품사태깅을 적용해줍니다.
train.head() # 데이터 확인

,index,id,reviews,target,tokenized_stem,main_pos
0,16441,16441,냄새도 사라지고 좋네요,5,냄새 도 사라지고 좋네요,냄새 도 사라지고 좋네요
1,18873,18873,필요한상품 한번에 구매할수 있어서 좋아요,5,필요한 상품 한번 에 구매 할수 있어서 좋아요,필요한 상품 한번 구매 할수 있어서 좋아요
2,10239,10239,배송 빨라서 좋구 맛있어서 좋아요 유통기한도 3월이라 생각보다 길어서 좋아요 그전에...,5,배송 빨 라서 좋구 맛있어서 좋아요 유통 기한 도 3월 이라 생각 보다 길어서 좋아...,배송 빨 라서 좋구 맛있어서 좋아요 유통 기한 도 3월 이라 생각 보다 길어서 좋아...
3,9690,9690,제모가 잘 되는지 모르겠네여,2,제모 가 잘 되는지 모르 겠네여,제모 가 잘 되는지 모르 겠네여
4,24695,24695,뭐에 홀렸는지 지난번에 뒤틀린 허니콤.. 약품냄새가 아직도 나는 실사 블라인드에 이...,1,뭐 에 홀렸는 지 지난번 에 뒤틀린 허니콤 .. 약품 냄새 가 아직도 나 는 실사 ...,뭐 홀렸는 지 지난번 뒤틀린 허니콤 약품 냄새 가 나 는 실사 블라인드 이어 어김 ...


#모델링

In [ ]:
from sklearn.model_selection import train_test_split

data = train.copy()
train, val = train_test_split(data)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해주었습니다.
val.reset_index(inplace=True)

In [ ]:
print( 'train 데이터 셋 모양 :', train.shape)
print( 'val 데이터 셋 모양 :', val.shape)

train 데이터 셋 모양 : (18750, 5)
val 데이터 셋 모양 : (6250, 5)


In [ ]:
train.head()

,index,id,reviews,target,tokenized_stem
0,16441,16441,냄새도 사라지고 좋네요,5,냄새 도 사라지고 좋네요
1,18873,18873,필요한상품 한번에 구매할수 있어서 좋아요,5,필요한 상품 한번 에 구매 할수 있어서 좋아요
2,10239,10239,배송 빨라서 좋구 맛있어서 좋아요 유통기한도 3월이라 생각보다 길어서 좋아요 그전에...,5,배송 빨 라서 좋구 맛있어서 좋아요 유통 기한 도 3월 이라 생각 보다 길어서 좋아...
3,9690,9690,제모가 잘 되는지 모르겠네여,2,제모 가 잘 되는지 모르 겠네여
4,24695,24695,뭐에 홀렸는지 지난번에 뒤틀린 허니콤.. 약품냄새가 아직도 나는 실사 블라인드에 이...,1,뭐 에 홀렸는 지 지난번 에 뒤틀린 허니콤 .. 약품 냄새 가 아직도 나 는 실사 ...


In [ ]:
#벡터화

X_train = train.main_pos #training 데이터에서 문서 추출
y_train = train.target #training 데이터에서 라벨 추출

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() #countvectorizer 생성
vectorizer.fit(X_train) # countvectorizer 학습
X_train_vec = vectorizer.transform(X_train) # transform

In [ ]:
from sklearn.linear_model import LogisticRegression #모델 불러오기
model = LogisticRegression(max_iter=500) #객체에 모델 할당
model.fit(X_train_vec, y_train) #모델 학습

LogisticRegression(max_iter=500)